In [1]:
"""This script imports necessary libraries for data processing and AWS S3 interaction.

- pandas (pd): Used for data manipulation and analysis.
- numpy (np): Used for advanced numerical operations and arrays.
- boto3: Provides an interface to interact with Amazon Web Services (AWS), including Amazon S3.
- StringIO: A module for working with in-memory string buffers, often used for reading data from or writing data to strings.
"""
import boto3
import pandas as pd
import numpy as np
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
s3 = boto3.resource('s3')

# Connecting to source S3 bucket
bucket = s3.Bucket('xetra-1234')

In [3]:
# Setting an argument date from which we need data into our dataframe "2022-12-25" in our case
arg_date = '2022-12-25'

# To get the data of previous day's closing price we will need to fetch data from 2022-12-24
arg_date_datetime = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days =1)
arg_date_datetime

datetime.date(2022, 12, 24)

In [4]:
# Getting a list of all the objects/files inside S3 bucket
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_datetime]

In [5]:
# We can see that list objects now contain details of all the files 
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-24/2022-12-24_BINS_XETR10.csv'),
 s3.Object

* we can see that "objects" now contains the list of all the files available inside the S3 Bucket named "xetra-1234" from date "2022-01-03" onwards

In [6]:
# Lets get a list of column names of the data
csv_obj_init = bucket.Object(key = objects[0].key).get().get('Body').read().decode('utf-8')
data_init = StringIO(csv_obj_init)
df_init = pd.read_csv(data_init, delimiter = ',')

In [7]:
# chechking the culumn names in dataframe
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [8]:
import warnings

# Filter out a specific warning
warnings.filterwarnings("ignore", category=FutureWarning)

# Creating an empty pandas dataframe and fixing the column names as per our dataset
df_merged = pd.DataFrame(columns=df_init.columns)

# Merging the data present in all the csv files into a single merged Dataset called df_merged using a for loop
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_merged = pd.concat([df_merged, df], ignore_index=True)

In [9]:
df_merged

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000A1J5RX9,O2D,TELEFONICA DTLD HLDG NA,Common stock,EUR,2504492,2022-12-24,08:00,2.433,2.444,2.433,2.438,183342,14
1,DE000A13SX22,HLE,HELLA GMBH+CO. KGAA O.N.,Common stock,EUR,2504580,2022-12-24,08:00,61.980,62.000,61.980,61.980,4415,7
2,DE0005102008,BSL,BASLER AG O.N.,Common stock,EUR,2504880,2022-12-24,08:00,108.800,108.800,108.800,108.800,1,1
3,DE0005103006,ADV,ADVA OPT.NETW.SE O.N.,Common stock,EUR,2504881,2022-12-24,08:00,14.500,14.540,14.500,14.500,5315,6
4,DE0005194005,BYW,BAYWA AG NA O.N.,Common stock,EUR,2504902,2022-12-24,08:00,49.200,49.200,49.200,49.200,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111171,GB00BLD4ZP54,CLTC,COINSHARES DIG.SEC.OEND,ETN,EUR,6479084,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,2
1111172,LU1923627332,RUSL,MUL-LYX.MSCI RUSSI.DIS.LS,ETF,EUR,5424594,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645,2
1111173,US98956P1021,ZIM,ZIMMER BIOMET HLDGS DL-01,Common stock,EUR,4582018,2022-12-31,20:30,113.100,113.100,113.100,113.100,0,1
1111174,US9224171002,VEO,"VEECO INSTRUMENTS DL-,01",Common stock,EUR,6198311,2022-12-31,20:30,24.600,24.600,24.600,24.600,0,1


In [10]:
# There are many columns which we do not need for further processing lets get rid of unwanted data
columns_to_included = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']

In [11]:
# Filtering our merged dataframe to only those columns which we need for further process
df_merged = df_merged.loc[:,columns_to_included]
df_merged

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000A1J5RX9,2022-12-24,08:00,2.433,2.444,2.433,2.438,183342
1,DE000A13SX22,2022-12-24,08:00,61.980,62.000,61.980,61.980,4415
2,DE0005102008,2022-12-24,08:00,108.800,108.800,108.800,108.800,1
3,DE0005103006,2022-12-24,08:00,14.500,14.540,14.500,14.500,5315
4,DE0005194005,2022-12-24,08:00,49.200,49.200,49.200,49.200,10
...,...,...,...,...,...,...,...,...
1111171,GB00BLD4ZP54,2022-12-31,16:46,19.324,19.324,19.324,19.324,0
1111172,LU1923627332,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645
1111173,US98956P1021,2022-12-31,20:30,113.100,113.100,113.100,113.100,0
1111174,US9224171002,2022-12-31,20:30,24.600,24.600,24.600,24.600,0


* df_merged now containes only those columns which we need for further analysis

In [12]:
# Lets check our dataset for any Null Values
df_merged.isna().sum()

ISIN            0
Date            0
Time            0
StartPrice      0
MaxPrice        0
MinPrice        0
EndPrice        0
TradedVolume    0
dtype: int64

## Data Transformation

In [13]:
# Lets check the dataframe to see whether we have any opening_price for the day
df_merged[(df_merged['ISIN'] == 'AT0000A0E9W5') & (df_merged['Date']== '2022-12-25')].sort_values(['Time'])

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
174258,AT0000A0E9W5,2022-12-25,08:00,14.02,14.27,14.02,14.27,1466
174435,AT0000A0E9W5,2022-12-25,08:02,14.27,14.29,14.27,14.29,351
174633,AT0000A0E9W5,2022-12-25,08:03,14.27,14.29,14.27,14.29,105
174784,AT0000A0E9W5,2022-12-25,08:04,14.29,14.29,14.29,14.29,40
176890,AT0000A0E9W5,2022-12-25,08:05,14.24,14.24,14.16,14.16,912
...,...,...,...,...,...,...,...,...
304253,AT0000A0E9W5,2022-12-25,16:21,13.91,13.96,13.90,13.96,4551
304807,AT0000A0E9W5,2022-12-25,16:23,13.98,13.99,13.98,13.98,709
306625,AT0000A0E9W5,2022-12-25,16:28,13.94,13.94,13.94,13.94,792
306951,AT0000A0E9W5,2022-12-25,16:29,13.93,13.93,13.92,13.92,975


* We can see that there is no column which provides us details about the opening price for a particular date
* Now in the next steps we will create a few columns like opening_price, closing_price etc, which will be a part of our data transformation process.

### Get Opening price per ISIN per day

In [14]:
# Creating a new column opening_price, which will be equal to the first reported StartPrice per ISIN per Day
df_merged['opening_price'] = df_merged.sort_values(['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
df_merged[(df_merged['ISIN'] == 'AT0000A0E9W5')]

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
2858,AT0000A0E9W5,2022-12-24,08:09,13.20,13.50,13.00,13.50,16763,13.20
3421,AT0000A0E9W5,2022-12-24,08:10,13.67,13.67,13.67,13.67,400,13.20
3789,AT0000A0E9W5,2022-12-24,08:11,13.71,13.88,13.71,13.88,1505,13.20
4127,AT0000A0E9W5,2022-12-24,08:12,13.90,13.92,13.90,13.92,500,13.20
4453,AT0000A0E9W5,2022-12-24,08:13,13.93,13.93,13.93,13.93,324,13.20
...,...,...,...,...,...,...,...,...,...
1106344,AT0000A0E9W5,2022-12-31,16:26,14.04,14.04,14.04,14.04,1129,13.88
1106720,AT0000A0E9W5,2022-12-31,16:27,14.03,14.14,14.02,14.04,6627,13.88
1107014,AT0000A0E9W5,2022-12-31,16:28,14.04,14.04,14.03,14.03,2003,13.88
1107365,AT0000A0E9W5,2022-12-31,16:29,14.08,14.08,14.00,14.00,354,13.88


* Now we have a column which shows the opening price per ISIN per day.
* We will now create some more columns in the same fashion which are needed for further analysis

### Get Closing price per ISIN per day

In [15]:
# Creating a new column named closing_price
df_merged['closing_price'] = df_merged.sort_values(['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')
df_merged[(df_merged['ISIN'] == 'AT0000A0E9W5')]

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
2858,AT0000A0E9W5,2022-12-24,08:09,13.20,13.50,13.00,13.50,16763,13.20,14.01
3421,AT0000A0E9W5,2022-12-24,08:10,13.67,13.67,13.67,13.67,400,13.20,14.01
3789,AT0000A0E9W5,2022-12-24,08:11,13.71,13.88,13.71,13.88,1505,13.20,14.01
4127,AT0000A0E9W5,2022-12-24,08:12,13.90,13.92,13.90,13.92,500,13.20,14.01
4453,AT0000A0E9W5,2022-12-24,08:13,13.93,13.93,13.93,13.93,324,13.20,14.01
...,...,...,...,...,...,...,...,...,...,...
1106344,AT0000A0E9W5,2022-12-31,16:26,14.04,14.04,14.04,14.04,1129,13.88,14.08
1106720,AT0000A0E9W5,2022-12-31,16:27,14.03,14.14,14.02,14.04,6627,13.88,14.08
1107014,AT0000A0E9W5,2022-12-31,16:28,14.04,14.04,14.03,14.03,2003,13.88,14.08
1107365,AT0000A0E9W5,2022-12-31,16:29,14.08,14.08,14.00,14.00,354,13.88,14.08


### Aggregations

In [16]:
df_merged = df_merged.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur = ('closing_price', 'min'), mimimum_price_eur = ('MinPrice','min'), maximum_price_eur = ('MaxPrice','max'), daily_traded_volume =('TradedVolume','sum'))

In [17]:
df_merged

,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-12-24,36.0500,36.1000,35.6500,37.0500,1838
1,AT000000STR1,2022-12-25,36.1000,37.7000,36.1000,37.7000,2864
2,AT000000STR1,2022-12-26,36.1000,37.7000,36.1000,37.7000,2864
3,AT000000STR1,2022-12-27,36.1000,37.7000,36.1000,37.7000,2864
4,AT000000STR1,2022-12-28,36.6000,36.7000,35.7500,36.7000,1773
...,...,...,...,...,...,...,...
25613,XS2434891219,2022-12-27,3.4404,3.5034,3.4404,3.5034,0
25614,XS2434891219,2022-12-28,3.4412,3.6620,3.4184,3.6620,0
25615,XS2434891219,2022-12-29,3.4412,3.6620,3.4184,3.6620,0
25616,XS2434891219,2022-12-30,3.4412,3.6620,3.4184,3.6620,0


* After the above transformations we have:
    1. Opening Price Column for the day
    2. Closing price column for the day
    3. maximum price column for the day
    4. minimum price column for the day
    5. all the above columns show amount in Euros
    6. Daily traded volumn for a particular stock for a particular day 

### Percent change previous closing

In [18]:
# creating a new column previous_closiong_price and assigning the value of closing_price_eur column of previous day
df_merged['previous_closing_price'] = df_merged.sort_values(['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

# Calculating precent change from previous day's closing price
df_merged['change_in_closing_in_%'] = round((df_merged['closing_price_eur'] - df_merged['previous_closing_price'])/df_merged['previous_closing_price']*100,2)

# Dropping column previous_closing_price as it is no longer needed
df_merged.drop(columns = ['previous_closing_price'], inplace = True)

# Lets round every number to upto two decimals
df_merged = df_merged.round(decimals = 2)
df_merged

,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume,change_in_closing_in_%
0,AT000000STR1,2022-12-24,36.05,36.10,35.65,37.05,1838,NaN
1,AT000000STR1,2022-12-25,36.10,37.70,36.10,37.70,2864,4.43
2,AT000000STR1,2022-12-26,36.10,37.70,36.10,37.70,2864,0.00
3,AT000000STR1,2022-12-27,36.10,37.70,36.10,37.70,2864,0.00
4,AT000000STR1,2022-12-28,36.60,36.70,35.75,36.70,1773,-2.65
...,...,...,...,...,...,...,...,...
25613,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
25614,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
25615,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
25616,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00


### Write the combined and aggregated dataset to a target S3 Bucket

In [19]:
# Describing the name of the key for output parquet file
key = 'xetra_daily_report' + datetime.today().strftime("%Y%m%d_%H%M%S")+'.parquet'

In [21]:
# Creating an in-memory buffer using BytesIO
out_buffer = BytesIO()

# Writing the DataFrame (df_merged) to the in-memory buffer in Parquet format
# with index=False (no index column in the Parquet file)
df_merged.to_parquet(out_buffer, index=False)

# Creating an S3 bucket object by specifying the bucket name
bucket_target = s3.Bucket('xetra-1234-etl-target-bucket')

# Uploading the Parquet data (from the in-memory buffer) to the S3 bucket
# by getting the buffer's content with getvalue() and specifying the key
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-1234-etl-target-bucket', key='xetra_daily_report20231022_204256.parquet')

### Reading the uploaded file to check whether the file content is correct

In [23]:
# reading the name of parquet file from S3 bucket
for obj in bucket_target.objects.all():
    print (obj.key)

xetra_daily_report20231022_204256.parquet


In [25]:
# Getting the S3 object ('prq_obj') from the specified S3 bucket ('bucket_target')
# The object key is set to 'xetra_daily_report20231022_204256.parquet'
# The object is retrieved from S3 and its body is read as bytes
prq_obj = bucket_target.Object(key='xetra_daily_report20231022_204256.parquet').get().get('Body').read()

# Create a BytesIO object ('data') and load the Parquet data ('prq_obj') into it
data = BytesIO(prq_obj)

# Read the Parquet data from the BytesIO object into a pandas DataFrame ('df_report')
df_report = pd.read_parquet(data)


,ISIN,Date,opening_price_eur,closing_price_eur,mimimum_price_eur,maximum_price_eur,daily_traded_volume,change_in_closing_in_%
0,AT000000STR1,2022-12-24,36.05,36.10,35.65,37.05,1838,NaN
1,AT000000STR1,2022-12-25,36.10,37.70,36.10,37.70,2864,4.43
2,AT000000STR1,2022-12-26,36.10,37.70,36.10,37.70,2864,0.00
3,AT000000STR1,2022-12-27,36.10,37.70,36.10,37.70,2864,0.00
4,AT000000STR1,2022-12-28,36.60,36.70,35.75,36.70,1773,-2.65
...,...,...,...,...,...,...,...,...
25613,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
25614,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
25615,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
25616,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
